<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/RFClassifier%2BRandomsearchCV_Bhavneesh_Churn_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
import numpy as np

import matplotlib.pyplot as plt


In [0]:
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [5]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
df.shape

(7043, 21)

In [7]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [8]:
df['customerID'].nunique()

7043

In [0]:
df = df.drop(columns="customerID")

In [10]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [11]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [0]:
#Many variables here are categorical and need to be encoded

In [12]:
df['MultipleLines'].nunique()

3

In [13]:
df['InternetService'].nunique()

3

In [14]:
df['Contract'].nunique()

3

In [15]:
df['Churn'].nunique()

2

In [0]:
#convert target column to binary 1 and 0 values

In [0]:
df['Churn'] = df['Churn'].replace(regex='Yes', value=1)
df['Churn'] = df['Churn'].replace(regex='No', value=0)

In [17]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [18]:
#Split into train and validation datasets

from sklearn.model_selection import train_test_split
train, val = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df['Churn'], random_state=42)
train.shape, val.shape

((5634, 20), (1409, 20))

In [19]:
train['Churn'].value_counts(normalize=True)

No     0.734647
Yes    0.265353
Name: Churn, dtype: float64

In [20]:
val['Churn'].value_counts(normalize=True)

No     0.734564
Yes    0.265436
Name: Churn, dtype: float64

In [0]:
#Thus, we have equal distribution of Churn in both train and val. and both datasets are balanced.

####Baseline target (churn): Zero features in train dataset

In [22]:
# Train dataset baseline
target = 'Churn'
y_train = train[target]
y_train.value_counts(normalize=True)

No     0.734647
Yes    0.265353
Name: Churn, dtype: float64

In [0]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)


In [24]:
#baseline accuracy if we guessed the majority class for every prediction on train dataset

from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

#Baseline accuracy with no features = 73.46%

0.7346467873624423

####RandomForestClassifier with RandomsearchCV

In [0]:
# 3. Arrange X feature matrices (already did y target vectors)
target = 'Churn'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]

In [0]:
#Apply encoding on train, validation and test datasets
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
train_encoded = encoder.fit_transform(X_train)
val_encoded = encoder.fit_transform(X_val)

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, preprocessing
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [0]:
# pipeline, identical to above but with ordinal encoder
pipeline = make_pipeline(   
    RandomForestClassifier(n_estimators=100, n_jobs=-1, max_features='auto', random_state=42),
)

In [0]:
#define parameter distributions for RandomizedSearchCV
param_distributions = { 
    'randomforestclassifier__n_estimators': randint(25, 1000), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, 30, None], 
}

In [30]:
%time
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(train_encoded, y_train)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 5.01 µs
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  50 | elapsed:    3.8s remaining:   23.2s
[Parallel(n_jobs=-1)]: Done  13 out of  50 | elapsed:    4.3s remaining:   12.2s
[Parallel(n_jobs=-1)]: Done  19 out of  50 | elapsed:    5.2s remaining:    8.5s
[Parallel(n_jobs=-1)]: Done  25 out of  50 | elapsed:    7.1s remaining:    7.1s
[Parallel(n_jobs=-1)]: Done  31 out of  50 | elapsed:    8.8s remaining:    5.4s
[Parallel(n_jobs=-1)]: Done  37 out of  50 | elapsed:    9.2s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  43 out of  50 | elapsed:    9.9s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   10.4s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('randomforestclassifier',
                                              RandomForestClassifier(bootstrap=True,
                                                                     ccp_alpha=0.0,
                                                                     class_weight=None,
                                                                     criterion='gini',
                                                                     max_depth=None,
                                                                     max_features='auto',
                                                                     max_leaf_nodes=None,
                                                                     max_samples=None,
                                                                     min_impurity_decrease=0.0,
                                           

In [31]:
print('Best hyperparameters', search.best_params_)
print('Best Train Accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__n_estimators': 58}
Best Train Accuracy 0.799962647813006


In [32]:
print('Best estimator', search.best_estimator_)

Best estimator Pipeline(memory=None,
         steps=[('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=10, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=58, n_jobs=-1,
                                        oob_score=False, random_state=42,
                                        verbose=0, warm_start=False))],
         verbose=False)


In [0]:
model = search.best_estimator_

In [0]:
y_pred = model.predict(val_encoded)

In [35]:
from sklearn.metrics import accuracy_score
val_accuracy = accuracy_score(y_val, y_pred)
print("RFClassifier RandomSearchCV hypertuning validation accuracy=", val_accuracy)

RFClassifier RandomSearchCV hypertuning validation accuracy= 0.7892122072391767


####GridSearch CV+ RandomForestClassifier

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


In [0]:
rfr = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_features='auto', random_state=42)

In [0]:
pipe = Pipeline([('rfr', rfr)])

In [0]:
param_grid = {
    'rfr__n_estimators': (25, 30, 35, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100),
    'rfr__max_depth': (15, 20, 25, 30, None)  
}


In [46]:
grid_search = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, verbose=1, scoring='accuracy', refit=True, return_train_score=True)
grid_search.fit(train_encoded, y_train)

Fitting 5 folds for each of 75 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:   12.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('rfr',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               max_samples=None,
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                                              

In [47]:
print('Best hyperparameters', grid_search.best_params_)
print('Best Train Accuracy', grid_search.best_score_)

Best hyperparameters {'rfr__max_depth': 15, 'rfr__n_estimators': 25}
Best Train Accuracy 0.7941043434131704


In [0]:
model2 = grid_search.best_estimator_

In [0]:
y_pred2 = model2.predict(val_encoded)

In [50]:
from sklearn.metrics import accuracy_score
val_accuracy = accuracy_score(y_val, y_pred2)
print("RFClassifier GridSearchCV hypertuning validation accuracy=", val_accuracy)

RFClassifier GridSearchCV hypertuning validation accuracy= 0.7842441447835344
